In [1]:
import numpy as np

import torch
import torch_geometric

seed = 12345

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Generacja zdarzeń
Na początku weźmy dla przykładu cztery zdarzenia dla pikselów XY = [[0, 0], [0, 1], [1, 0], [1, 1]], z czasami T [0, 0.1, 0.2, 0.3] oraz atrybutami P [5, 6, 7, 8]. Generujemy tensor zdarzeń [ev0, ev1, ev2, ev3].

In [2]:
# Tworzenie 4 różnych zdarzeń
ev0 = [0, 0, 0, 5]
ev1 = [0, 1, 0.1, 6]
ev2 = [1, 0, 0.2, 7]
ev3 = [1, 1, 0.3, 8]

# Tworzenie wektora/tensora zdarzeń
events = torch.tensor([ev0, ev1, ev2, ev3])
print(events)

tensor([[0.0000, 0.0000, 0.0000, 5.0000],
        [0.0000, 1.0000, 0.1000, 6.0000],
        [1.0000, 0.0000, 0.2000, 7.0000],
        [1.0000, 1.0000, 0.3000, 8.0000]])


Z każdego zdarzenia tworzymy wierzchołki grafu o pozycji pos = [X, Y, T] oraz wartości x = [P]. Zapisujemy to w formacie Data z biblioteki PyTorch Geometric.

In [3]:
from torch_geometric.data import Data

x, pos = events[:, -1:], events[:, :3]
data = Data(x=x, pos=pos)

print('Cały format danych: ', data)
print('Atrybuty wierzchołków: ', data.x)
print('Pozycje wierzchołków: ', data.pos)

Cały format danych:  Data(x=[4, 1], pos=[4, 3])
Atrybuty wierzchołków:  tensor([[5.],
        [6.],
        [7.],
        [8.]])
Pozycje wierzchołków:  tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.1000],
        [1.0000, 0.0000, 0.2000],
        [1.0000, 1.0000, 0.3000]])


Teraz "generujemy" krawędzie pomiędzy wierzchołkami/zdarzeniami. Na początku tworzymy graf skierowany, gdzie zdarzenia ev1, ev2 oraz ev3 są skierowane w stronę zdarzenia ev0. Oznacza to, że informacje będą propagowane od wierzchołków ev1, ev2, ev3 do ev0 -> tylko ev0 zmieni swoją wartość x.

Pierwszy wiersz odnosi się do indeksu wierzchołka, z którego wychodzi krawędź, a drugi wiersz wierzchołek docelowy. Indeksy odnoszą się do wektora x oraz pos.

In [4]:
edge_index = torch.tensor([[1, 2, 3], [0, 0, 0]])
data.edge_index = edge_index

print('Cały format danych: ', data)
print('Indeksy sąsiedztwa: ', data.edge_index)

Cały format danych:  Data(x=[4, 1], pos=[4, 3], edge_index=[2, 3])
Indeksy sąsiedztwa:  tensor([[1, 2, 3],
        [0, 0, 0]])


# Warstwy konwolucyjne - PointNetConv

Ogólna zasada działania warstw konwolucyjnych na grafie określa propagację informacji do wierzchołka grafu na podstawie wartości sąsiadujących wierzchołków oraz wartości krawędzi ich łączących (obie wartości są opcjonalne). Konwolucja definiowana jest w 3 funkcjach:
- message function - tworzy ona wiadomość (informację) dla dla każdego sąsiadującego wierzchołka, która następnie zostanie przesłana do aktualnego wierzchołka.
- aggregation function - na podstawie wiadomości (informacji) uzyskanych przez wszystkich sąsiadów, agreguje je do jednej wartości, która zostanie wykorzystana do aktualizacji wierzchołka. Przeważnie są to funkcje max, min, mean z wszystkich wartości od sąsiadów.
- update function - aktualizuje ona informację danego wierzchołka na podstawie zagregowanych informacji.

Jest to ogólny zarys działania konwolucji na grafie. Możliwe są różne wariancje tych funkcji, mogą one być opcjonalne, oraz wykorzystane informację zależą od reprezentacji.

Jest wiele różnych stron, które to dokładniej tłumaczą:
- https://danielegrattarola.github.io/posts/2021-03-12/gnn-lecture-part-2.html
- https://pytorch-geometric.readthedocs.io/en/latest/tutorial/create_gnn.html

Teraz tworzymy klasę warstwy konwolucyjnej. My raczej będziemy korzystać z konwolucji PointNetConv - jest ona prosta, szybka, dostosowana do przetwarzania danych typu PointCloud. Taka warstwa PointNetConv przyjmuje 3 argumenty:
- local_nn - jest to warstwa wejściowa typu MLP, która mapuje dane wejściowe sąsiada równe (x oraz różnicy pos sąsiada względem docelowego wierzchołka
- global_nn - jest to warstwa wyjściowa typu MLP, przetwarza dane po funckji agregacji, które nadpiszą x sąsiada
- add_self_loop - dodaje do wektora sąsiedztwa dla każdego wierzchołka połączenie ze sobą, tzn (idx_ev0, idx_ev0)

Dokładny opis tej konwolucji jest w dokumentacji:
https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.PointNetConv.html

oraz jego implementacja w pythonie:
https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/conv/point_conv.html#PointNetConv

In [5]:
from torch.nn import Linear
from torch.nn.functional import relu
from torch_geometric.nn.conv import PointNetConv
from torch_geometric.nn.norm import BatchNorm

class PointNetBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, add_loop):
        super().__init__()

        self.local_nn = Linear(in_channels + 3, out_channels)
        self.conv = PointNetConv(self.local_nn, add_self_loops=add_loop)
        self.norm = BatchNorm(in_channels = out_channels)

    def forward(self, x, pos, edge_index):
        x = self.conv(x, pos, edge_index)

        # Na wyjściu takiej warstwy mamy aktualizację wartości x. Można ją poddać aktywacji funkcji oraz normalizacji:
        # x = relu(x)      # Activation
        # x = self.norm(x) # Normalization 
        return x

Tworzymy pojedynczą warstwę konwolucyjną przy założeniu, że nie dodajemy self_loops. Możemy określić ilość wejść (pierwsza warstwa ma tylko jedną cechę x dla każdego wierzchołka) oraz wyjściową ilość cech. Na razie ustalimy przejście z 1 -> 1.

Podajemy jako wejście sieci wartości x, pos oraz krawędzie edge_index grafu wejściowego. W wyniku dostajemu zaktualizowane wartości x.

In [8]:
conv = PointNetBlock(1, 1, False)

new_x = conv(data.x, data.pos, data.edge_index)

print(data.x)
print(new_x)

tensor([[5.],
        [6.],
        [7.],
        [8.]])
tensor([[-2.1661],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]], grad_fn=<ScatterReduceBackward0>)


Tutaj spróbujemy zrobić dokładnie to samo, ale z przejściem 1 -> 4:

In [9]:
conv = PointNetBlock(1, 4, False)

new_x = conv(data.x, data.pos, data.edge_index)

print(data.x)
print(new_x)

tensor([[5.],
        [6.],
        [7.],
        [8.]])
tensor([[-3.0979, -0.5534, -1.2027,  0.8512],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000]],
       grad_fn=<ScatterReduceBackward0>)


Ponieważ krawędzie tworzą graf skierowany, gdzie wierzchołek pierwszy, drugi i trzeci skierowane są w stronę wierzchołka zerowego, tylko wierzchołek zerowy ma aktualizowaną wartość. 

Implementacja takich warstw najpierw generuje wektor wyjściowy zerowy a następnie go aktualizuje odpowiednio na podstawie wyniku konwolucji dla wierzchołkow, które mają sąsiadów. Ponieważ pozostałe wierzchołki nie mają żadnego sąsiada, aby mogłaby zostać zaktualizowana wartość, pozostają one zerowe.

Można spróbować ręcznie zduplikować wartości, aby nie były zerami, tylko wartościami przed konwolucją:

In [10]:
new_x[1:, :] = data.x[1:] # Uwzględniamy tylko indeksy tych wierzchołków, które nie były zmienione
print(new_x)

tensor([[-3.0979, -0.5534, -1.2027,  0.8512],
        [ 6.0000,  6.0000,  6.0000,  6.0000],
        [ 7.0000,  7.0000,  7.0000,  7.0000],
        [ 8.0000,  8.0000,  8.0000,  8.0000]], grad_fn=<CopySlices>)


W przypadku self_loop = True nie ma takiego problemu, ponieważ każdy wierzchołek będzie miał minimum jednego sąsiada - samego siebie.

In [11]:
conv = PointNetBlock(1, 4, True)

new_x = conv(data.x, data.pos, data.edge_index)

print(data.x)
print(new_x)

tensor([[5.],
        [6.],
        [7.],
        [8.]])
tensor([[ 2.4808, -1.2056, -0.7681, -2.4180],
        [ 1.3959, -1.5472, -0.8319, -2.8610],
        [ 1.6172, -1.7461, -0.8958, -3.3039],
        [ 1.8385, -1.9450, -0.9596, -3.7469]],
       grad_fn=<ScatterReduceBackward0>)


# Generacja grafu - rzeczywiste dane